In [18]:
import numpy as np
import pandas as pd
import imp
import os
import sys

# import tensorflow as tf
import pickle
from numba import njit
from functools import lru_cache
from IPython.display import Audio, display
from sklearn.preprocessing import OrdinalEncoder
import random
import torch
import pytorch_lightning

# from matplotlib import pyplot as plt

import stops
import make_loaders

# tf.random.set_seed(1234)

imp.reload(stops)
imp.reload(make_loaders)


%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


# notes
* we should check if we can move state between different lengths of sequences, if so we can train on many differnent bus lines.

In [19]:
url_exception = "http://www.wav-sounds.com/movie/austinpowers.wav"
url_exception = "https://www.myinstants.com/media/sounds/roblox-death-sound_1.mp3"


def play_sound(self, etype, value, tb, tb_offset=None):
    self.showtraceback((etype, value, tb), tb_offset=tb_offset)
    display(Audio(url=url_exception, autoplay=True))


get_ipython().set_custom_exc((Exception,), play_sound)

url_done = (
    "https://sound.peal.io/ps/audios/000/000/537/original/woo_vu_luvub_dub_dub.wav"
)
url_done = "https://www.myinstants.com/media/sounds/taco-bell-bong-sfx.mp3"
url_done = "https://www.myinstants.com/media/sounds/magic_immune.mp3"


def allDone():
    display(
        Audio(
            url=url_done,
            autoplay=True,
        )
    )

In [20]:
data_dir = os.path.join(os.getcwd(), "..", "data")

In [21]:
direction = 1

In [22]:
def get_data(recompute, direction):
    def df_pop(df, labels):
        popped = df[labels]
        df = df.drop(columns=labels)
        return df, popped

    data_dir = os.path.join(os.getcwd(), "..", "data")

    trips, dates, labels = None, None, None
    lstm_dir = os.path.join(data_dir, "trips_for_lstm")
    files = [
        f"{f}_direction_{direction}.npy"
        for f in ["non_categories", "categories", "labels", "cardinality"]
    ]

    cardinality = []

    if recompute:

        pickle_path = os.path.join(data_dir, "mega_pickle")
        df = pd.read_pickle(pickle_path)
        #     df = df.head(100000)

        df = df.drop(
            columns=["route_number", "time_cat_stop_time", "time_cat_leave_time"]
        )

        df = df.sort_values(["service_date", "train", "trip_number", "stop_time"])
        df = df.reset_index(drop=True)

        df = df[df["direction"] == direction]

        dtypes = df.dtypes
        category_names = dtypes[dtypes == "category"]
        category_names = category_names.index.to_list()
        non_category_names = dtypes[dtypes != "category"]
        non_category_names = non_category_names.index.to_list()
        non_category_names.remove("service_date")
        non_category_names.remove("arrival_deviance")
        label_names = ["arrival_deviance"]

        enc = OrdinalEncoder()
        df[category_names] = enc.fit_transform(df[category_names])
        df[category_names] = df[category_names].astype(np.int32)

        for category in category_names:
            cardinality.append(len(df[category].unique()))

        trips = df.groupby(["service_date", "train", "trip_number"])
        trips = [trip for _, trip in trips]

        random.shuffle(
            trips
        )  # WE CAN DO THIS BECAUSE WE'RE ONLY TRYING TO PREDICT THE NEXT TRIP

        hist = {}
        for trip in trips:
            trip = trip.drop_duplicates("location_id")
            s = tuple(trip["location_id"].to_list())
            if s in hist:
                hist[s] = hist[s] + 1
            else:
                hist[s] = 1

        sorted_stops = sorted(hist.items(), key=lambda x: x[1], reverse=True)
        most_common_stop_sequence = list(sorted_stops[0][0])[
            1:-1
        ]  # LAST STOP WAS REGULARLY ~13 MINUTES EARLY...
        cannonical_length = len(most_common_stop_sequence)

        stop_order_dict = {}
        for i, stop in enumerate(most_common_stop_sequence):
            stop_order_dict[stop] = i

        x1 = []
        x2 = []
        y = []
        for trip in trips:
            trip = trip.drop_duplicates("location_id")

            labels = trip[label_names]
            categories = trip[category_names]
            non_categories = trip[non_category_names]

            cats = np.array(categories)
            non_cats = np.array(non_categories)
            l = np.array(labels)

            c_cats = np.zeros((cannonical_length, cats.shape[1]))
            c_non_cats = np.zeros((cannonical_length, non_cats.shape[1]))
            c_l = np.zeros(cannonical_length)

            loc_id_index = category_names.index("location_id")

            this_trips_stops = cats[:, loc_id_index].tolist()

            for i, loc_id in enumerate(most_common_stop_sequence):
                if loc_id in this_trips_stops:
                    idx = this_trips_stops.index(loc_id)
                    c_cats[i] = cats[idx]
                    c_non_cats[i] = non_cats[idx]
                    c_l[i] = l[idx]

            x1.append(c_non_cats)
            x2.append(c_cats)
            y.append(c_l)

        non_categories = np.stack(x1)
        categories = np.stack(x2)
        labels = np.stack(y)
        cardinality = np.array(cardinality)

        arrays = [non_categories, categories, labels, cardinality]

        for file_name, array in zip(files, arrays):
            file_path = os.path.join(lstm_dir, file_name)
            np.save(file_path, array)

    else:
        arrays = []
        for file_name in files:
            file_path = os.path.join(lstm_dir, file_name)
            arrays.append(np.load(file_path))
        non_categories, categories, labels, cardinality = arrays

    return non_categories, categories, labels, cardinality


# non_categories, categories, labels, cardinality = get_data(direction=1, recompute=False)


# print(
#     non_categories.shape,
#     categories.shape,
#     labels.shape,
# )


# datalen = non_categories.shape[0]
# num_stops = non_categories.shape[1]
# non_category_width = non_categories.shape[2]
# category_width = categories.shape[2]

In [23]:
class bus_dataset(torch.utils.data.Dataset):
    def __init__(self, recompute, direction, batch_size):

        self.non_categories, self.categories, self.labels, self.cardinality = get_data(
            recompute, direction
        )

        self.batch_size = batch_size

        self.datalen = self.non_categories.shape[0]
        self.num_stops = self.non_categories.shape[1]
        self.non_category_width = self.non_categories.shape[2]
        self.category_width = self.categories.shape[2]

        self.non_categories_shape = (self.num_stops, self.non_category_width)
        self.categories_shape = (self.num_stops, self.category_width)
        self.label_shape = (self.num_stops,)

    def __len__(self):
        return self.datalen

    def __getitem__(self, idx):
        r_int = random.randint(0, self.num_stops)
        r = torch.tensor(r_int).float()
        non_category = torch.tensor(self.non_categories[idx]).float()
        category = torch.tensor(self.categories[idx]).float()
        label = torch.tensor([self.labels[idx]]).float()

        # MIGHT OVERWRITE MEMORY WITH ZEROS, CHECK THAT TORCH.TENSOR MAKES NEW MEMORY
        #         r_unpacked = r[0]
        #         print(r_unpacked)

        non_category[r_int:] = 0
        category[r_int:] = 0

        label = torch.transpose(label, 0, 1)

        #         print('label', label.shape)
        #         print('non_category', non_category.shape)
        #         print('category', category.shape)

        return (non_category, category, r), label


batch_size = 32

dataset = bus_dataset(recompute=False, direction=1, batch_size=batch_size)

train = 0.6
val = 0.3
test = 0.1
assert round(train + val + test) == 1

datalen = len(dataset)

train = int(train * datalen)
val = int(val * datalen)
test = datalen - (train + val)

print(train, val, test)


train_data, test_data = torch.utils.data.random_split(dataset, [train + val, test])
train_data, val_data = torch.utils.data.random_split(train_data, [train, val])


train_loader, val_loaders, test_loader = make_loaders.make_loaders(
    train_data, val_data, test_data, batch_size
)
#     train_loader = torch.utils.data.DataLoader(
#         train_data,
#         batch_size=batch_size,
#         shuffle=True,
#         num_workers=12,
#     )
#     val_loader = torch.utils.data.DataLoader(
#         val_data,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=12,
#     )
#     test_loader = torch.utils.data.DataLoader(
#         test_data,
#         batch_size=batch_size,
#         shuffle=False,
#         num_workers=12,
#     )

print("made dataloaders")

27521 13760 4588
made dataloaders


In [24]:
drop_rate = 0.2
lstm_size = 32


class LSTM(pytorch_lightning.core.lightning.LightningModule):
    def __init__(self, data):
        super().__init__()

        print("init model")

        self.data = data

        self.lstm_1 = torch.nn.LSTM(
            self.data.non_category_width, lstm_size, batch_first=True, num_layers=3
        )
        self.lstm_to_outs = torch.nn.Linear(lstm_size, 1)

        # https://github.com/PyTorchLightning/pytorch-lightning/discussions/6748#discussioncomment-568785

    def forward(self, x):
        non_category, category, r = x
        x, state = self.lstm_1(non_category)
        x = self.lstm_to_outs(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = torch.nn.functional.mse_loss(y_hat, y)
        return loss

    def configure_optimizers(self):
        print("configure opt")
        return torch.optim.Adam(self.parameters(), lr=0.02)

In [ ]:
model = LSTM(data=dataset)
trainer = pytorch_lightning.Trainer(gpus=1)
print("made trainer")
trainer.fit(model, train_loader)
print("done fitting")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


init model
made trainer
configure opt



  | Name         | Type   | Params
----------------------------------------
0 | lstm_1       | LSTM   | 23.4 K
1 | lstm_to_outs | Linear | 33    
----------------------------------------
23.5 K    Trainable params
0         Non-trainable params
23.5 K    Total params
0.094     Total estimated model params size (MB)


Epoch 0:   0%|                                                                                                                                                                  | 0/861 [00:00<00:08, 98.14it/s]

In [ ]:
drop_rate = 0.2
lstm_size = 32

# https://stackoverflow.com/questions/56729139/faster-way-to-do-multiple-embeddings-in-pytorch

non_categories_input = tf.keras.Input(shape=non_categories_shape, name="non_categories")
categories_input = tf.keras.Input(shape=categories_shape, name="categories")
# mask_input = tf.keras.Input(shape=label_shape, name="mask")

non_categories = non_categories_input
categories = categories_input
# mask = mask_input

cat_list = tf.unstack(categories, axis=2)
embed_out = []
for cat, card in zip(cat_list, cardinality):
    embed_out.append(tf.keras.layers.Embedding(card, card // 3 + 1)(cat))

lstm_prep = tf.keras.layers.Concatenate(axis=2)([non_categories, *embed_out])

concat_width = lstm_prep.shape[2]

lstm_prep = tf.keras.layers.Dense((lstm_size + concat_width) // 2 + 1)(lstm_prep)
lstm_prep = tf.keras.layers.Dense(lstm_size)(lstm_prep)
block_out = tf.keras.layers.Reshape((num_stops, lstm_size))(lstm_prep)


lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

lstm_layers = tf.keras.layers.BatchNormalization()(lstm_layers)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

block_out = tf.keras.layers.add([block_out, lstm_layers])

lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

lstm_layers = tf.keras.layers.BatchNormalization()(lstm_layers)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

block_out = tf.keras.layers.add([block_out, lstm_layers])

lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

lstm_layers = tf.keras.layers.BatchNormalization()(lstm_layers)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

block_out = tf.keras.layers.add([block_out, lstm_layers])


lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

lstm_layers = tf.keras.layers.BatchNormalization()(lstm_layers)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

block_out = tf.keras.layers.add([block_out, lstm_layers])


lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

lstm_layers = tf.keras.layers.BatchNormalization()(lstm_layers)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)
lstm_layers = tf.keras.layers.LSTM(lstm_size, return_sequences=True)(lstm_layers)

block_out = tf.keras.layers.add([block_out, lstm_layers])


lstm_layers = tf.keras.layers.BatchNormalization()(block_out)
lstm_layers = tf.keras.layers.Dropout(rate=drop_rate)(lstm_layers)

out_funnel = tf.keras.layers.Dense(1)

pred = tf.keras.layers.TimeDistributed(out_funnel)(lstm_layers)

pred = tf.keras.layers.Reshape(label_shape)(pred)
# pred = pred * mask

model = tf.keras.Model(
    inputs=[non_categories_input, categories],
    outputs=[pred],
)

In [ ]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.MeanSquaredError(reduction="auto", name="mean_squared_error"),
    metrics=["MAE"],
)
print("total parameters: {:,}".format(model.count_params()))
tf.keras.utils.plot_model(model, "model.png", show_shapes=True, dpi=227)

In [ ]:
monitor_param = "val_loss"

nan_terminate = tf.keras.callbacks.TerminateOnNaN()

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor=monitor_param, patience=10  # , restore_best_weights=True
)

model_dir = os.path.join(data_dir, "model")
checkpoint_filepath = os.path.join(model_dir, "checkpoint")
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor=monitor_param,
    mode="min",
    save_best_only=True,
)

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))

model.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset,
    callbacks=[early_stopping, checkpoint, nan_terminate],
)

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
# model.evaluate(test_dataset)

In [ ]:
preds = model.predict(test_dataset)

In [ ]:
print(preds.shape)

In [ ]:
for i, (pred, y) in enumerate(zip(preds, test_dataset.unbatch().as_numpy_iterator())):
    if i < 100:
        x, y = y
        pred_len = pred.shape[0]

        num_samples = x["num_samples"]
        samp = [0] * num_samples

        x = list(range(pred_len))
        #         print(pred)

        plt.rcParams["figure.figsize"] = [20, 4]
        plt.scatter(x, pred, label="pred")
        plt.scatter(x, y, label="y")
        plt.scatter(list(range(num_samples)), samp, label="samp")
        plt.legend()
        plt.show()

allDone()